In [1]:
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import re
from itertools import product
from pprint import pp
from math import ceil
from snowflake.snowpark.session import Session

In [2]:
professions = ['Dentist', 'CPA', 'Plumber']
locations = ['Olathe,KS', 'Overland Park, KS', 'Leawood,KS', 'Kansas City,KS', 'Kansas City,MO', 'Lees Summit,MO', 'Independence,MO', 'Blue Springs,MO']


In [3]:
params = {'search_terms': 'CPA',
          'geo_location_terms': quote_plus('Lees Summit,MO')}

In [4]:
resp = requests.get('http://yellowpages.com/search', params=params)

In [5]:
content = resp.content
soup = BeautifulSoup(content, 'html.parser')
soup = BeautifulSoup(content, 'html.parser')
res = soup.findAll(class_ = 'info')
soup.find_all(class_ = 'showing-count')[0]

<span class="showing-count">Showing 1-30 of 434<span class="result-info">More info</span></span>

In [20]:
bus_name = r">[A-Z a-z0-9.,&;\-]+<"
bus_add = r">[A-Z a-z0-9.,&;]+<"
bus_phone = r">[0-9() -]+<"

In [7]:
temp = re.findall(bus_phone,str(res[2].findAll(class_ = 'phones phone primary')))
re.sub(r'[<>() -]', '', str(temp))



"['8162514575']"

In [9]:
connection_parameters = {}
connection_parameters["account"] = 'qnzkqqi-prod'
connection_parameters["database"] = 'SANDBOXES'
connection_parameters["schema"] = 'MBJORNSON'
connection_parameters["warehouse"] = 'COMPUTE_WH'
connection_parameters["role"] = 'SYSADMIN'
connection_parameters["user"] = 'mbjornson'
connection_parameters["Authenticator"] = "externalbrowser"
session = Session.builder.configs(connection_parameters).create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://ffbkc.okta.com/app/snowflake/exka3eb1paQUAt90P4x7/sso/saml?SAMLRequest=lZJRb9owGEX%2FSuQ9J05CWFsLqAKoKxvQAAFNfTOJAx6OnflzCO2vnwll6h5aaW%2BRc6597Pv17k%2BlcI5MA1eyjwLPRw6Tmcq53PXROn1wb5EDhsqcCiVZH70wQPeDHtBSVCSuzV4u2e%2BagXHsRhJI%2B6OPai2JosCBSFoyICYjq3g2JaHnk0orozIl0LvI5wkKwLSxhtdIDtzq7Y2pCMZN03hNx1N6h0Pf97F%2Fhy11Rr5c%2BZO90wd8gP3ozFvC4smb25DLyxN8prW9QEAe0zRxk6dVipz4qjpSEuqS6RXTR56x9XJ6EQBrsJg%2F%2F1gsJm6yfBp7IFVTCHpgmSqr2tgNPfuFC5ZjoXbc3nky7qPqwHM9P%2BYi%2Byb1PupOU7oZZq%2Fhz83jLzqMlebFqMO%2Fzr7Hs1F%2Bu86Qs7mWGp5LnQDUbCLPVRq75IeRG%2Fhu2E2DDvG7pBN5NzfRM3LGtkouqWmTV9%2Bi2B4yTx0MbdVoVeG%2F1pidDrTDtkFFF%2BvY3PlJdLrBAAqfe0WXUSHt8XrwHw%2FQw%2B%2BDbwM3tx1MxokSPHtxHpQuqfm4osAL2hWeu0WLElZSLuI81wzAViWEakaaUWPn2uiaITy4nPrvZA%2F%2BAA%3D%3D&Relay

In [71]:
temp = pd.DataFrame([{'BUSINESS_NAME': re.sub(r'[<>]', '',re.findall(bus_name,str(x.findAll(class_ = 'business-name')))[0]) if re.findall(bus_name,str(x.findAll(class_ = 'business-name'))) else '' ,
                 'STREET': re.sub(r'[<>]', '',re.findall(bus_add,str(x.findAll(class_ = 'street-address')))[0]) if re.findall(bus_add,str(x.findAll(class_ = 'street-address'))) else '' ,
                 'LOCALITY':re.sub(r'[<>]', '',re.findall(bus_add,str(x.findAll(class_ = 'locality')))[0]) if re.findall(bus_add,str(x.findAll(class_ = 'locality'))) else '' ,
                 'PHONE':re.sub(r'[<>() -]', '',re.findall(bus_phone,str(x.findAll(class_ = 'phones phone primary')))[0]) if re.findall(bus_phone,str(x.findAll(class_ = 'phones phone primary'))) else '', 
                 'SEARCH_TERM': params['search_terms']
                } for x in res if x.findAll(class_ = 'street-address') != []])

In [66]:

temp['city'] = temp['LOCALITY'].apply(lambda x: x.split(',')[0])
temp['state'] = temp['LOCALITY'].apply(lambda x: x.split(',')[1].split()[0])
temp['zip'] = temp['LOCALITY'].apply(lambda x: x.split(',')[1].split()[1])

In [69]:
temp['LOCALITY'] = temp['LOCALITY'].str.split(',')
temp['city'] = temp['LOCALITY'].apply(lambda x: x[0])
temp['state'] = temp['LOCALITY'].apply(lambda x: x[1].split()[0])
temp['zip'] = temp['LOCALITY'].apply(lambda x: x[1].split()[1])


In [80]:
temp['LOCALITY'] = temp['LOCALITY'].apply(lambda s: re.sub(r'(?<=, [A-Z]{2}) ', ', ', s))
temp['LOCALITY'] = temp['LOCALITY'].str.split(',')


In [33]:

def YP_SCRAPER(session:Session, professions:list, locations:list) -> pd.DataFrame:
    searches = [(x,y) for x in professions for y in locations]
    results = []
    df = pd.DataFrame()
    for s in searches:
        params = {'search_terms': quote_plus(s[0]),
          'geo_location_terms': quote_plus(s[1]), 
          'page' : 1
        }
        results = []
        resp = requests.get('http://yellowpages.com/search', params=params)
        content = resp.content
        soup = BeautifulSoup(content, 'html.parser')
        res_ct = ceil(int(re.findall("(?<=of )[0-9]+", str(soup.find_all(class_ = 'showing-count')))[0])/30) if re.findall("(?<=of )[0-9]+", str(soup.find_all(class_ = 'showing-count')))[0] != [] else 0
        #int(re.findall("(?<=[0-9]-)[0-9]+", str(soup.find_all(class_ = 'showing-count')))[0])
        while params['page'] < 4: #res_ct:
          if params['page'] >1:
            resp = requests.get('http://yellowpages.com/search', params=params)
            content = resp.content
            soup = BeautifulSoup(content, 'html.parser')
          res = soup.findAll(class_ = 'info')
          params['page'] +=1
          #print(params['page'])
          results += res
        temp = pd.DataFrame([{'business_name': re.sub(r'[<>]', '',re.findall(bus_name,str(x.findAll(class_ = 'business-name')))[0]) if re.findall(bus_name,str(x.findAll(class_ = 'business-name'))) else '' ,
                 'street_address': re.sub(r'[<>]', '',re.findall(bus_add,str(x.findAll(class_ = 'street-address')))[0]) if re.findall(bus_add,str(x.findAll(class_ = 'street-address'))) else '' ,
                 'locality':re.sub(r'[<>]', '',re.findall(bus_add,str(x.findAll(class_ = 'locality')))[0]) if re.findall(bus_add,str(x.findAll(class_ = 'locality'))) else '' ,
                 'phone':re.sub(r'[<>() -]', '',re.findall(bus_phone,str(x.findAll(class_ = 'phones phone primary')))[0]) if re.findall(bus_phone,str(x.findAll(class_ = 'phones phone primary'))) else '', 
                 'search_term': params['search_terms']
                } for x in res if x.findAll(class_ = 'street-address') != []])
        df = pd.concat([df, temp])
    df.to_snowflake('test_table', if_exists='replace', index=False)
    return df

In [ ]:
df = YP_SCRAPER(session, professions, locations)

In [21]:
df.to_snowflake('test_table', if_exists='replace', index=False)

In [ ]:
session.sql(''' 
            select 
            *
            from sandboxes.information_schema.stages'''
            ).collect()

In [126]:
def yp_scrapper_proc(session:Session, professions:list, locations:list) -> str:
    import pandas
    df = YP_SCRAPER(professions, locations)
    session.write_pandas(df, 'YP_SCRAPE', overwrite=True)
    return 'finished'

In [ ]:
yp_scrapper_sproc = session.sproc.register(func=yp_scrapper_proc, 
                                             name='yp_scrapper_sproc', 
                                             is_permanent=True, 
                                             replace=True, 
                                             stage_location='@SNDBX', 
                                             packages= ['snowflake-snowpark-python', 'pandas', 'itertools','joblib'])